In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
from keras import layers, models, Sequential
from keras.layers import Input, Conv2D, SeparableConv2D,MaxPooling2D, Dense, Flatten, Reshape, Dropout, BatchNormalization, Activation, GlobalAveragePooling2D, GlobalMaxPool2D
from tensorflow.keras.utils import plot_model
import os
import shutil
import random
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, CSVLogger
from tensorflow.keras import layers
import math
from matplotlib import pyplot

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [ ]:
INIT_LR               = 0.01 
WEIGHT_DECAY          = 1e-5
CLIP_THRESHOLD        = 1.0
SEED=42
MAX_EPOCH = 250
BATCH_SIZE = 32
IMG_SIZE = (224, 224)

def set_seeds(seed=SEED):
    random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)
set_seeds()

In [ ]:
class WarmUpCosine(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(
        self, learning_rate_base, total_steps, warmup_learning_rate, warmup_steps
    ):
        super(WarmUpCosine, self).__init__()

        self.learning_rate_base = learning_rate_base
        self.total_steps = total_steps
        self.warmup_learning_rate = warmup_learning_rate
        self.warmup_steps = warmup_steps
        self.pi = tf.constant(np.pi)
        
    def get_config(self):
        config = {
        'learning_rate_base': self.learning_rate_base,
        'total_steps': self.total_steps,
        'warmup_learning_rate': self.warmup_learning_rate,
        'warmup_steps': self.warmup_steps,
         }
        return config

    def __call__(self, step):
        if self.total_steps < self.warmup_steps:
            raise ValueError("Total_steps must be larger or equal to warmup_steps.")
        learning_rate = (
            0.5
            * self.learning_rate_base
            * (
                1
                + tf.cos(
                    self.pi
                    * (tf.cast(step, tf.float32) - self.warmup_steps)
                    / float(self.total_steps - self.warmup_steps)
                )
            )
        )

        if self.warmup_steps > 0:
            if self.learning_rate_base < self.warmup_learning_rate:
                raise ValueError(
                    "Learning_rate_base must be larger or equal to "
                    "warmup_learning_rate."
                )
            slope = (
                self.learning_rate_base - self.warmup_learning_rate
            ) / self.warmup_steps
            warmup_rate = slope * tf.cast(step, tf.float32) + self.warmup_learning_rate
            learning_rate = tf.where(
                step < self.warmup_steps, warmup_rate, learning_rate
            )
        return tf.where(
            step > self.total_steps, 0.0, learning_rate, name="learning_rate"
        )


In [ ]:
kaiming_normal = keras.initializers.VarianceScaling(scale=2.0, mode='fan_out', distribution='untruncated_normal')

def conv3x3(x, out_planes, stride=1, dilation_rate=1, name=None):
    x = layers.ZeroPadding2D(padding=1, name=f'{name}_pad')(x)
    return layers.Conv2D(filters=out_planes, kernel_size=3, strides=stride, use_bias=False, kernel_initializer=kaiming_normal, name=name)(x)
 
def eca_block(inputs, b=1, gama=2):
    
    #  Enter the number of channels of the characteristic graph 
    in_channel = inputs.shape[-1]
    
    #  Calculate the size of adaptive convolution kernel according to the formula 
    kernel_size = int(abs((math.log(in_channel, 2) + b) / gama))
    
    #  If the convolution kernel size is even , Just use it 
    if kernel_size % 2:
        kernel_size = kernel_size
    
    #  If the convolution kernel size is odd, it becomes even 
    else:
        kernel_size = kernel_size + 1
    
    # [h,w,c]==>[None,c]  Global average pooling 
    x = layers.GlobalAveragePooling2D()(inputs)
    
    # [None,c]==>[c,1]
    x = layers.Reshape(target_shape=(in_channel, 1))(x)
    
    # [c,1]==>[c,1]
    x = layers.Conv1D(filters=1, kernel_size=kernel_size, padding='same', use_bias=False)(x)
    
    # sigmoid Activate 
    x = tf.nn.sigmoid(x)
    
    # [c,1]==>[1,1,c]
    x = layers.Reshape((1,1,in_channel))(x)
    
    #  The result is multiplied by the input 
    outputs = layers.multiply([inputs, x])
    
    return outputs



def basic_block(x, planes, stride=1, downsample=None, name=None):
    identity = x

    out = layers.Conv2D(filters=planes, kernel_size=3, strides=1, padding='same',use_bias=False, kernel_initializer=kaiming_normal, name = f'{name}.conv1')(x)
    out = layers.BatchNormalization(momentum=0.9, epsilon=1e-5, name=f'{name}.bn1')(out)
    out = tf.keras.activations.swish(out)
    
    out = layers.Conv2D(filters=planes, kernel_size=3, strides=stride, padding='same',use_bias=False, kernel_initializer=kaiming_normal, name = f'{name}.conv2')(out)
    out = layers.BatchNormalization(momentum=0.9, epsilon=1e-5, name=f'{name}.bn2')(out)
    
    #Channel Attention - ECA
    out = eca_block(out)

    if downsample is not None:
        for layer in downsample:
            identity = layer(identity)
    
    #skip connection
    out = layers.Add(name=f'{name}.add')([identity, out])
    out = tf.keras.activations.swish(out)

    return out

def make_layer(x, planes, blocks, stride=1, name=None):
    downsample = None
    inplanes = x.shape[3]
    if stride != 1 or inplanes != planes:
        downsample = [
            layers.Conv2D(filters=planes, kernel_size=1, strides=stride, use_bias=False, kernel_initializer=kaiming_normal, name=f'{name}.0.downsample.0'),
            layers.BatchNormalization(momentum=0.9, epsilon=1e-5, name=f'{name}.0.downsample.1'),
        ]

    x = basic_block(x, planes, stride, downsample, name=f'{name}.0')
    for i in range(1, blocks):
        x = basic_block(x, planes, name=f'{name}.{i}')

    return x


def custom(x, blocks_per_layer, num_classes=2):
    x = layers.Rescaling(1. / 255.)(x)
    x = layers.ZeroPadding2D(padding=3, name='conv1_pad')(x)
    x = layers.Conv2D(filters=64, kernel_size=7, strides=2, use_bias=False, kernel_initializer=kaiming_normal, name='conv1')(x)
    x = layers.BatchNormalization(momentum=0.9, epsilon=1e-5, name='bn1')(x)
    x = tf.keras.activations.swish(x)
    x = layers.ZeroPadding2D(padding=1, name='maxpool_pad')(x)
    x = layers.MaxPool2D(pool_size=3, strides=2, name='maxpool')(x)

    x = make_layer(x, 64, blocks_per_layer[0], name='layer1')
    x = make_layer(x, 128, blocks_per_layer[1], stride=2, name='layer2')
    x = make_layer(x, 256, blocks_per_layer[2], stride=2, name='layer3')
    x = make_layer(x, 512, blocks_per_layer[3], stride=2, name='layer4')

    x = layers.GlobalAveragePooling2D(name='avgpool')(x)
    initializer = keras.initializers.RandomUniform(-1.0 / math.sqrt(512), 1.0 / math.sqrt(512))
    x = layers.Dense(units=num_classes,activation='softmax', kernel_initializer=initializer, bias_initializer=initializer, name='fc')(x)

    return x

def CustomArchi(x):
    return custom(x, [2, 2, 2, 2])


In [ ]:
inputs = keras.Input(shape=(224, 224, 3))
outputs = CustomArchi(inputs)
model = keras.Model(inputs, outputs)
model.summary()

In [ ]:
ARCHITECTURE = 'CustomArchitecture'

BASE_DIR = '/tf/DAPPEM/multi_fold_data_V2'
MODEL_SAVE_DIRECTORY_PAR =  '/tf/DAPPEM/saved-models-dv2'
if not os.path.exists(MODEL_SAVE_DIRECTORY_PAR):
    os.mkdir(MODEL_SAVE_DIRECTORY_PAR)
    print(MODEL_SAVE_DIRECTORY_PAR)
MODEL_SAVE_DIRECTORY =  os.path.join(MODEL_SAVE_DIRECTORY_PAR, ARCHITECTURE)

if os.path.exists(MODEL_SAVE_DIRECTORY):
    shutil.rmtree(MODEL_SAVE_DIRECTORY, ignore_errors=True)
os.mkdir(MODEL_SAVE_DIRECTORY)


In [ ]:
for i in range(1,6):
    fold_name =  'fold'+str(i)
    print(fold_name,'........')
    fold_dir = os.path.join(BASE_DIR, fold_name)  
    train_dir = os.path.join(fold_dir, 'train_dir_aug')
    test_dir = os.path.join(BASE_DIR, 'test_dir')
    validation_dir = os.path.join(fold_dir, 'val_dir')
    
    train_dataset = image_dataset_from_directory(train_dir,
                                                 shuffle=True,
                                                 batch_size=BATCH_SIZE,
                                                 image_size=IMG_SIZE)

    validation_dataset = image_dataset_from_directory(validation_dir,
                                                      shuffle=True,
                                                      batch_size=BATCH_SIZE,
                                                      image_size=IMG_SIZE)
    AUTOTUNE = tf.data.AUTOTUNE
    train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
    validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)

    inputs = keras.Input(shape=(224, 224, 3))
    outputs = resnet18(inputs)
    model = keras.Model(inputs, outputs)
    
    TOTAL_STEPS = len(train_dataset)* MAX_EPOCH
    warmup_steps = int(0.05*TOTAL_STEPS)

    scheduled_lrs = WarmUpCosine(learning_rate_base=INIT_LR,
                                 total_steps=TOTAL_STEPS,
                                 warmup_learning_rate=0.0,
                                 warmup_steps=warmup_steps)

    model.compile(optimizer=tfa.optimizers.AdamW(weight_decay=WEIGHT_DECAY, 
                                learning_rate=scheduled_lrs,
                                clipnorm=CLIP_THRESHOLD),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics='accuracy')
    early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=20)
    
    model_name = ARCHITECTURE + fold_name + '.h5'
    model_save_path = os.path.join(MODEL_SAVE_DIRECTORY, model_name)
    
    mc = ModelCheckpoint(model_save_path, monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

    csv_logger = CSVLogger(os.path.join(MODEL_SAVE_DIRECTORY,"model_history_log_fold"+str(i)+".csv"), append=True)
    history_bef_finetune = model.fit(train_dataset, epochs=MAX_EPOCH, validation_data=validation_dataset, callbacks=[mc, csv_logger])
    pyplot.plot(history_bef_finetune.history['loss'], label='train')
    pyplot.plot(history_bef_finetune.history['val_loss'], label='test')
    pyplot.legend()
    pyplot.show()
    

In [ ]:
import sys
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
sys.path.insert(1, 'utils')
from summarizeModelALLMetricPercPMSMCUSTOM import summarizeModel
summarizeModel(IMG_SIZE = (224, 224),
                  ARCHITECTURE = 'CustomArchitecture')